In [1]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc read-aloud"><h2>--- Day 18: Like a GIF For Your Yard ---</h2><p>After the <a href="6">million lights incident</a>, the fire code has gotten stricter: now, at most <span title="This is an outrage!  We're going to the next town hall meeting.">ten thousand lights</span> are allowed.  You arrange them in a 100x100 grid.</p>
<p>Never one to let you down, Santa again mails you instructions on the ideal lighting configuration.  With so few lights, he says, you'll have to resort to <em>animation</em>.</p>
<p>Start by setting your lights to the included initial configuration (your puzzle input).  A <code>#</code> means "on", and a <code>.</code> means "off".</p>
<p>Then, animate your grid in steps, where each step decides the next configuration based on the current one.  Each light's next state (either on or off) depends on its current state and the current states of the eight lights adjacent to it (including diagonals).  Lights on the edge of the grid might have fewer than eight neighbors; the missing ones always count as "off".</p>
<p>For example, in a simplified 6x6 grid, the light marked <code>A</code> has the neighbors numbered <code>1</code> through <code>8</code>, and the light marked <code>B</code>, which is on an edge, only has the neighbors marked <code>1</code> through <code>5</code>:</p>
<pre><code>1B5...
234...
......
..123.
..8A4.
..765.
</code></pre>
<p>The state a light should have next is based on its current state (on or off) plus the <em>number of neighbors that are on</em>:</p>
<ul>
<li>A light which is <em>on</em> stays on when <code>2</code> or <code>3</code> neighbors are on, and turns off otherwise.</li>
<li>A light which is <em>off</em> turns on if exactly <code>3</code> neighbors are on, and stays off otherwise.</li>
</ul>
<p>All of the lights update simultaneously; they all consider the same current state before moving to the next.</p>
<p>Here's a few steps from an example configuration of another 6x6 grid:</p>
<pre><code>Initial state:
.#.#.#
...##.
#....#
..#...
#.#..#
####..
<br/>
After 1 step:
..##..
..##.#
...##.
......
#.....
#.##..

After 2 steps:
..###.
......
..###.
......
.#....
.#....

After 3 steps:
...#..
......
...#..
..##..
......
......

After 4 steps:
......
......
..##..
..##..
......
......
</code></pre>

<p>After <code>4</code> steps, this example has four lights on.</p>
<p>In your grid of 100x100 lights, given your initial configuration, <em>how many lights are on after 100 steps</em>?</p>
</article>


In [2]:
from functools import total_ordering
from typing import Generator


grid = """
.#.#.#
...##.
#....#
..#...
#.#..#
####..
"""


class Grid:
    directions = (
        (0, 1),
        (1, 1),
        (1, 0),
        (1, -1),
        (0, -1),
        (-1, -1),
        (-1, 0),
        (-1, 1),
    )

    def __init__(self, grid: str | list[list[str]]) -> None:
        if isinstance(grid, list):
            self.grid = grid
        else:
            self.grid = self.parse(grid)
        self.rows = len(self.grid)
        self.cols = len(self.grid[0])

    def parse(self, grid):
        return [list(row) for row in grid.strip().splitlines()]

    def __repr__(self) -> str:
        return "\n".join("".join(row) for row in self.grid)

    def neigbors(self, row: int, col: int) -> Generator[tuple[int, int]]:
        return (
            (row + dr, col + dc)
            for dr, dc in self.directions
            if 0 <= row + dr < self.rows and 0 <= col + dc < self.cols
        )

    def step(self):
        new_grid = [["." for _ in range(self.cols)] for _ in range(self.rows)]

        for row, col in product(range(self.rows), range(self.cols)):
            count_on = sum(
                1 for r, c in self.neigbors(row, col) if self.grid[r][c] == "#"
            )
            if self.grid[row][col] == "#":
                # A light which is on stays on when 2 or 3 neighbors are on,
                #  and turns off otherwise.
                if count_on == 2 or count_on == 3:
                    new_grid[row][col] = "#"
                else:
                    new_grid[row][col] = "."
            else:
                # A light which is off turns on if exactly 3 neighbors are on,
                # and stays off otherwise.
                if count_on == 3:
                    new_grid[row][col] = "#"
                else:
                    new_grid[row][col] = "."

        return Grid(new_grid)

    def count_lights_on(self) -> int:
        return sum(
            1
            for r, c in product(range(self.rows), range(self.cols))
            if self.grid[r][c] == "#"
        )

    def __eq__(self, other: Grid) -> bool:
        return self.grid == other.grid

    @classmethod
    def steps(cls, grid: Grid, n: int) -> Grid:
        for _ in range(n):
            grid = grid.step()
        return grid

In [3]:
expected = [
    # initial state
    """
.#.#.#
...##.
#....#
..#...
#.#..#
####..
""",
    # after 1 step
    """
..##..
..##.#
...##.
......
#.....
#.##..
""",
    # after 2 steps
    """
..###.
......
..###.
......
.#....
.#....
""",
    # after 3 steps
    """
...#..
......
...#..
..##..
......
......
""",
    # after 4 steps
    """
......
......
..##..
..##..
......
......
""",
]

expected = [Grid(g) for g in expected]

actual = [None] * 4

actual[0] = Grid(grid)
for i in range(1, 4):
    actual[i] = actual[i - 1].step()


assert all(a == e for a, e in zip(actual, expected))

In [4]:
g1 = Grid(grid)
assert g1.count_lights_on() == 15
ge = Grid.steps(g1, 4)
assert ge == expected[4]
assert ge.count_lights_on() == 4

In [5]:
with open("../input/day18.txt") as f:
    gf = Grid(f.read())

gf100 = Grid.steps(gf, 100)
gf100.count_lights_on()

821

<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>821</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>You flip the instructions over; Santa goes on to point out that this is all just an implementation of <a href="https://en.wikipedia.org/wiki/Conway's_Game_of_Life">Conway's Game of Life</a>.  At least, it was, until you notice that something's wrong with the grid of lights you bought: four lights, one in each corner, are <em>stuck on</em> and can't be turned off.  The example above will actually run like this:</p>
<pre><code>Initial state:
##.#.#
...##.
#....#
..#...
#.#..#
####.#

After 1 step:
#.##.#
####.#
...##.
......
#...#.
#.####

After 2 steps:
#..#.#
#....#
.#.##.
...##.
.#..##
##.###

After 3 steps:
#...##
####.#
..##.#
......
##....
####.#

After 4 steps:
#.####
#....#
...#..
.##...
#.....
#.#..#

After 5 steps:
##.###
.##..#
.##...
.##...
#.#...
##...#
</code></pre>

<p>After <code>5</code> steps, this example now has <code>17</code> lights on.</p>
<p>In your grid of 100x100 lights, given your initial configuration, but with the four corners always in the <em>on</em> state, <em>how many lights are on after 100 steps</em>?</p>
</article>
</main>


In [6]:
import enum
from typing import override


class GridII(Grid):
    @override
    def parse(self, grid):
        grid = [list(row) for row in grid.strip().splitlines()]
        rows, cols = len(grid), len(grid[0])
        grid[0][0] = "#"
        grid[0][cols - 1] = "#"
        grid[rows - 1][0] = "#"
        grid[rows - 1][cols - 1] = "#"
        return grid

    @override
    def step(self):
        new_grid = [["." for _ in range(self.cols)] for _ in range(self.rows)]

        for row, col in product(range(self.rows), range(self.cols)):
            count_on = sum(
                1 for r, c in self.neigbors(row, col) if self.grid[r][c] == "#"
            )
            if row == 0 and col == 0:
                new_grid[row][col] = "#"
            elif row == 0 and col == self.cols - 1:
                new_grid[row][col] = "#"
            elif row == self.rows - 1 and col == 0:
                new_grid[row][col] = "#"
            elif row == self.rows - 1 and col == self.cols - 1:
                new_grid[row][col] = "#"
            elif self.grid[row][col] == "#":
                # A light which is on stays on when 2 or 3 neighbors are on,
                #  and turns off otherwise.
                if count_on == 2 or count_on == 3:
                    new_grid[row][col] = "#"
                else:
                    new_grid[row][col] = "."
            else:
                # A light which is off turns on if exactly 3 neighbors are on,
                # and stays off otherwise.
                if count_on == 3:
                    new_grid[row][col] = "#"
                else:
                    new_grid[row][col] = "."

        return GridII(new_grid)


expected_str = [
    # Initial state:
    """
##.#.#
...##.
#....#
..#...
#.#..#
####.#
""",
    # After 1 step:
    """
#.##.#
####.#
...##.
......
#...#.
#.####
""",
    # After 2 steps:
    """
#..#.#
#....#
.#.##.
...##.
.#..##
##.###
""",
    # After 3 steps:
    """
#...##
####.#
..##.#
......
##....
####.#
""",
    # After 4 steps:
    """
#.####
#....#
...#..
.##...
#.....
#.#..#
""",
    # After 5 steps:
    """
##.###
.##..#
.##...
.##...
#.#...
##...#
""",
]
expected = [GridII(g) for g in expected_str]

actual = [None] * 5

actual[0] = GridII(expected_str[0])
for i in range(1, 5):
    actual[i] = actual[i - 1].step()


assert all(a == e for a, e in zip(actual, expected))

In [7]:
g2 = GridII(expected_str[0])
assert g2.count_lights_on() == 17
ge = GridII.steps(g2, 5)
assert ge == expected[5]
assert ge.count_lights_on() == 17

In [8]:
with open("../input/day18.txt") as f:
    gf = GridII(f.read())

gf100 = GridII.steps(gf, 100)
gf100.count_lights_on()

886

<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>886</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>
<p>At this point, you should <a href="/2015">return to your Advent calendar</a> and try another puzzle.</p>
<p>If you still want to see it, you can <a href="18/input" target="_blank">get your puzzle input</a>.</p>
<p>You can also <span class="share">[Share<span class="share-content">on
  <a href="https://twitter.com/intent/tweet?text=I%27ve+completed+%22Like+a+GIF+For+Your+Yard%22+%2D+Day+18+%2D+Advent+of+Code+2015&amp;url=https%3A%2F%2Fadventofcode%2Ecom%2F2015%2Fday%2F18&amp;related=ericwastl&amp;hashtags=AdventOfCode" target="_blank">Twitter</a>
  <a href="javascript:void(0);" onclick="var ms; try{ms=localStorage.getItem('mastodon.server')}finally{} if(typeof ms!=='string')ms=''; ms=prompt('Mastodon Server?',ms); if(typeof ms==='string' &amp;&amp; ms.length){this.href='https://'+ms+'/share?text=I%27ve+completed+%22Like+a+GIF+For+Your+Yard%22+%2D+Day+18+%2D+Advent+of+Code+2015+%23AdventOfCode+https%3A%2F%2Fadventofcode%2Ecom%2F2015%2Fday%2F18';try{localStorage.setItem('mastodon.server',ms);}finally{}}else{return false;}" target="_blank">Mastodon</a></span>]</span> this puzzle.</p>
</main>
